In [19]:
import pickle
import pandas as pd
import numpy as np
from tqdm import tqdm

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [5]:
import os
os.chdir('/Users/sonjiyeong/Documents/랩실 프로젝트/스마트폰 리뷰마이닝/output')

In [2]:
import tensorflow as tf

# GPU 사용 설정
device_name = tf.test.gpu_device_name()
if not tf.config.list_physical_devices('GPU'):
    print("GPU를 찾을 수 없습니다. 런타임 유형을 GPU로 변경해주세요.")
else:
    print(f"GPU 장치: {device_name}")

GPU 장치: /device:GPU:0


2024-05-19 23:06:39.848576: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2024-05-19 23:06:39.848600: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-05-19 23:06:39.848605: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-05-19 23:06:39.848685: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-05-19 23:06:39.848944: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


# 1. 훈련 데이터셋 구축
- 목적 : 이후 APC (속성 기반 감성 예측) 모델이 속성을 식별할 수 있도록 속성 단어 앞뒤에 문자 ([B-ASP], [E-ASP]) 추가
- 예시 : battery life is great for me -> [B-ASP]battery life[E-ASP] is great for me
- 처리 프로세스
    1. 토큰화된 리스트의 단어 중 속성 사전에 있는 단어가 포함되는지 확인
    2. 속성 사전에 정의된 단어인 경우 단어의 앞뒤에 [B-ASP], [E-ASP] 추가
    3. 토큰화된 리스트를 띄어쓰기로 이어서 문장으로 재구성 (APC 모델에서 문장을 토큰화해주는 기능이 포함되어 있기 때문)

In [6]:
df_dic = pickle.load(open('속성사전_데이터프레임.pkl', 'rb'))

In [7]:
def count_underscores(text):
    return text.count('_')

df_dic['ngram'] = df_dic['pre_aspect'].apply(count_underscores)
df_dic = df_dic.sort_values(by='ngram', ascending=False).reset_index(drop=True)

In [8]:
df_dic.head(3)

,aspect,ori_aspect,pre_aspect,count,all_cnt,feature,simil_feature,ngram
0,ease of use screen size screen resolution came...,ease_of_use_screen_size_screen_resolution_came...,eas_use_screen_size_screen_resolut_camera_resolut,1,1,NaN,dis,7
1,3 . 5 mm headphone jack adapter,3_5_mm_headphone_jack_adapter,3_5_mm_headphon_jack_adapt,1,1,NaN,acc,5
2,air view air gesture smart,air_view_air_gesture_smart,air_view_air_gestur_smart,1,1,NaN,func,4


In [9]:
data = pickle.load(open('감성분석후_속성전처리_데이터프레임.pkl', 'rb'))
print('데이터 개수:', data.shape[0])
df = data.copy().reset_index(drop=True)
df.head(2)

데이터 개수: 76215


,id,model,date,rating,sentence,IOB,tokens,aspect,position,sentiment,probs,confidence,new_tokens
0,1,galaxy_s3,2012-05-11,2,too plasticky for $ 700 . When I put a review ...,"['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ...","[too, plasticky, for, $, 700, ., when, i, put,...","[costs, materials]","[[45], [52]]","[Negative, Negative]","[[0.9986786246299744, 0.000984302954748273, 0....","[0.9987, 0.9983]","[too, plasticky, for, $, 700, ., when, i, put,..."
1,2,galaxy_s3,2012-06-01,5,So far super good . I got my S3 in white yeste...,"['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ...","[so, far, super, good, ., i, got, my, s3, in, ...","[memory, battery, battery life]","[[48], [57], [59, 60]]","[Neutral, Positive, Positive]","[[0.0006813739310018718, 0.5115107893943787, 0...","[0.5115, 0.9756, 0.9979]","[so, far, super, good, ., i, got, my, s3, in, ..."


In [71]:
def space_bar(word_list):
    combined_words = list(word_list)
    for compound in df_dic['aspect'].tolist():
        words = compound.split()
        compound_word = ' '.join(words)  
        
        idx = 0
        while idx < len(combined_words):
            if combined_words[idx:idx+len(words)] == words:
                combined_words[idx:idx+len(words)] = [compound_word]
                idx += len(words)
            else:
                idx += 1
    return combined_words

In [72]:
with tf.device(device_name):
    df['pre_tokens'] = df['tokens'].apply(space_bar)

In [74]:
lst_mark_sentence = []
lst_ori_feature = []
lst_feature = []
lst_main = []
lst_id = []

with tf.device(device_name):
    for row in range(df.shape[0]):
        lst_tokens = list(df.loc[row, 'pre_tokens'])


        for idx in range(len(lst_tokens)):

            if lst_tokens[idx] in df_dic['aspect'].tolist():
                compound = lst_tokens[idx]
                lst_new = lst_tokens[:]
                lst_new[idx] = '[B-ASP]' + lst_tokens[idx] + '[E-ASP]'
                mark_sentence = " ".join(lst_new)
                lst_mark_sentence.append(mark_sentence)
                lst_id.append(df.loc[row, 'id'])
                lst_ori_feature.append(df_dic.loc[df_dic['aspect'] == compound, 'aspect'].values[0])
                lst_feature.append(df_dic.loc[df_dic['aspect'] == compound, 'pre_aspect'].values[0])
                lst_main.append(df_dic.loc[df_dic['aspect'] == compound, 'simil_feature'].values[0])

In [75]:
df_apc_dataset = pd.DataFrame({'id':lst_id, 'mark_sentence':lst_mark_sentence, 
                               'ori_feature':lst_ori_feature, 'feature':lst_feature, 'main_feature':lst_main,})

In [76]:
print('데이터 개수:', df_apc_dataset.shape[0])
df_apc_dataset.head()

데이터 개수: 102927


,id,mark_sentence,ori_feature,feature,main_feature
0,1,too plasticky for $ 700 . when i put a review ...,quality materials,qualiti_materi,body
1,2,so far super good . i got my s3 in white yeste...,memory,memori,stor
2,2,so far super good . i got my s3 in white yeste...,battery,batteri,bat
3,2,so far super good . i got my s3 in white yeste...,battery life,batteri_life,bat
4,3,expensive or respect . i ' ve narrowed my choi...,16gb,16gb,stor


In [78]:
df_apc_dataset.to_csv('apc_훈련데이터셋.csv', index=False)

# 2. Aspect Sentiment Classification
- **pyabsa** : [🔗깃허브 주소](https://github.com/yangheng95/PyABSA/tree/v2), 
[🔗Documentation](https://pyabsa.readthedocs.io/en/latest/index.html)

- **aspect sentiment classification** : [🔗깃허브 주소](https://github.com/yangheng95/PyABSA/blob/v2/examples-v2/aspect_polarity_classification/Aspect_Sentiment_Classification.ipynb)   
</br>
- **모델 설명**
    - 속성 기반 감성 예측 모델
    - 지정한 속성 (ex: [B-ASP]battery life[E-ASP])에 대한 감성을 '긍정, 부정, 중립', 3가지로 표현함
    - 한 문장에 여러 개의 속성이 존재하는 경우 각 속성에 대한 감성을 각각 도출

In [ ]:
!pip install pyabsa -U

In [ ]:
!pip install pyabsa==v1.16.27

In [ ]:
from pyabsa import AspectPolarityClassification as APC, available_checkpoints

# you can view all available checkpoints by calling available_checkpoints()
checkpoint_map = available_checkpoints(show_ckpts=True)

classifier = APC.SentimentClassifier('english',
                                     auto_device=True,  # False means load model on CPU
                                     cal_perplexity=True,
                                     )

In [ ]:
'''# instance inference
classifier.predict(['I love this movie, it is so great!'],
                   save_result=True,
                   print_result=True,  # print the result
                   ignore_error=True,  # ignore the error when the model cannot predict the input
                   )

inference_source = APC.APCDatasetList.Laptop14
apc_result = classifier.batch_predict(target_file=inference_source,  #
                                      save_result=True,
                                      print_result=True,  # print the result
                                      pred_sentiment=True,  # Predict the sentiment of extracted aspect terms
                                      )

print(apc_result)'''

In [11]:
data = pd.read_csv('apc_훈련데이터셋.csv')

In [12]:
print('데이터 개수:', data.shape[0])
data.head()

데이터 개수: 102927


,id,mark_sentence,ori_feature,feature,main_feature
0,1,too plasticky for $ 700 . when i put a review ...,quality materials,qualiti_materi,body
1,2,so far super good . i got my s3 in white yeste...,memory,memori,stor
2,2,so far super good . i got my s3 in white yeste...,battery,batteri,bat
3,2,so far super good . i got my s3 in white yeste...,battery life,batteri_life,bat
4,3,expensive or respect . i ' ve narrowed my choi...,16gb,16gb,stor


In [13]:
data.loc[3, 'mark_sentence']

"so far super good . i got my s3 in white yesterday i had the iphone 4s before but i played before with the s2 that i already liked . the phone feels great in hand it ' s super fast . i like the fact to expand memory by micro sd and that you can change battery . [B-ASP]battery life[E-ASP] is great for me so far looking forward to explore more features . in case you have the htc one x i think you don ' t need to upgrade otherwise i can highly recommend it it ' s not feeling so huge than the note ."

In [ ]:
df = data.iloc[:40000, :]
df_text = df['mark_sentence'].tolist()

with tf.device(device_name):
    lst_result1 = []
    idx = df.index[0]
    for ex in df_text:
        result = classifier.predict(
            text=ex,
            print_result=False,
            ignore_error=True,  # ignore an invalid example, if it is False, invalid examples will raise Exceptions
            eval_batch_size=32,
        )
        lst_result1.append(result)
        print('############## idx:', idx, '##############')
        idx += 1

file_name = 'apc_result_1.pkl'

# 데이터를 pickle 파일로 저장
with open(file_name, 'wb') as file:
    pickle.dump(lst_result1, file)

In [ ]:
df = data.iloc[40000:80000, :]
df_text = df['mark_sentence'].tolist()

with tf.device(device_name):
    lst_result2 = []
    idx = df.index[0]
    for ex in df_text:
        result = classifier.predict(
            text=ex,
            print_result=False,
            ignore_error=True,  # ignore an invalid example, if it is False, invalid examples will raise Exceptions
            eval_batch_size=32,
        )
        lst_result2.append(result)
        print('############## idx:', idx, '##############')
        idx += 1

file_name = 'apc_result_2.pkl'

# 데이터를 pickle 파일로 저장
with open(file_name, 'wb') as file:
    pickle.dump(lst_result2, file)

In [ ]:
df = data.iloc[80000:, :]
df_text = df['mark_sentence'].tolist()


with tf.device(device_name):
    lst_result3 = []
    idx = df.index[0]
    for ex in tqdm(df_text):
        result = classifier.predict(
            text=ex,
            print_result=False,
            ignore_error=True,  # ignore an invalid example, if it is False, invalid examples will raise Exceptions
            eval_batch_size=32,
        )
        lst_result3.append(result)
        print('############## idx:', idx, '##############')
        idx += 1

file_name = 'apc_result_3.pkl'

# 데이터를 pickle 파일로 저장
with open(file_name, 'wb') as file:
    pickle.dump(lst_result3, file)

In [ ]:
lst_1 = pickle.load(open('apc_result_1.pkl', 'rb'))
lst_2 = pickle.load(open('apc_result_2.pkl', 'rb'))
lst_3 = pickle.load(open('apc_result_3.pkl', 'rb'))

df_1 = pd.DataFrame(lst_1)
df_2 = pd.DataFrame(lst_2)
df_3 = pd.DataFrame(lst_3)

df_all = pd.concat([df_1, df_2, df_3]).reset_index(drop=True)

In [20]:
df_all.head()
df_all.shape

,text,aspect,sentiment,confidence,probs,ref_sentiment,ref_check,perplexity
0,too plasticky for $ 700 . when i put a review ...,['quality materials'],['Negative'],[0.9467122554779053],"[array([0.94671226, 0.05119575, 0.00209207], d...",['-100'],[''],1.161164
1,so far super good . i got my s3 in white yeste...,['memory'],['Positive'],[0.9818015694618225],"[array([0.00167393, 0.01652445, 0.98180157], d...",['-100'],[''],1.160901
2,great in hand it ' s super fast . i like the f...,['battery'],['Positive'],[0.9737927317619324],"[array([0.00311911, 0.02308813, 0.97379273], d...",['-100'],[''],1.161632
3,great in hand it ' s super fast . i like the f...,['battery life'],['Positive'],[0.985840916633606],"[array([0.00214477, 0.01201428, 0.9858409 ], d...",['-100'],[''],1.161632
4,expensive or respect . i ' ve narrowed my choi...,['16gb'],['Negative'],[0.4394790828227997],"[array([0.43947908, 0.33674648, 0.22377446], d...",['-100'],[''],1.160578


(102927, 8)

In [16]:
df_all['sentiment'].value_counts()

['Negative']    64265
['Positive']    34159
['Neutral']      4503
Name: sentiment, dtype: int64

In [ ]:
import pickle
file_name = '감성예측결과_데이터프레임.pkl'

# 데이터를 pickle 파일로 저장
with open(file_name, 'wb') as file:
    pickle.dump(df_all, file)

df_all.to_csv('감성예측결과_데이터프레임.csv', index=False)